In [3]:
import time
from AaltoAtoms import CircCorral, get_latman_dest, get_atom_that_moved_the_most
from AMRL import Createc_Controller
import numpy as np

In [2]:
CC = Createc_Controller()

# get atom positions at beginning
Cim = np.array(CC.stm.scandata(1,4))
zconst = float(CC.stm.getparam('ZPiezoConst'))
nmx = nmy = CC.get_len_nm()
C = CircCorral(Cim, zconst, nmx, nmy)
C.subtract_plane()
C.get_region_centroids(percentile=98, edge_cutoff=0.1, show=False)
central_atom_loc = C.get_central_atom(C.centroids)

old_centroids = C.centroids
original_circle = C.r, C.c

# manipulation destinations are always safe_dist_nm from the edge of the circle
safe_dist_nm = 1.2

# parameters for lateral manipulation
latman_millivolt = 3
latman_picoamp = 80000

# ramp to this bias before scanning
scan_bias_mv = 40

# ramp to this bias before taking spectra
spec_bias_mv = 40


offset_nm = CC.get_xy_nm()
len_nm = CC.get_len_nm()

# take the first spectrum at the central atom location
CC.ramp_bias_mV(spec_bias_mv)
CC.stm.btn_vertspec(*central_atom_loc)
CC.stm.vertsave()

while True:
    # get a destination within the circle to move the atom 
    dest = get_latman_dest(original_circle, C, safe_dist_nm)
    
    # convert to nm 
    init = pixel_to_nm(central_atom_loc, Cim, offset_nm, [nmx, nmy])
    final = pixel_to_nm(dest, Cim, offset_nm, [nmx, nmy])

    # do the manipulation
    latmandata = CC.lat_manipulation(*init, *final, latman_millivolt, latman_picoamp, offset_nm, nmx)
    
    # scan corral and save scan
    CC.ramp_bias_mV(scan_bias_mv)
    CC.stm.scanstart()
    CC.stm.scanwaitfinished()
    time.sleep(1)
    CC.stm.quicksave()
    
    # get position of atom that moved the most since last scan
    central_atom_loc_old, original_circle_old = central_atom_loc, original_circle
    central_atom_loc, original_circle = get_atom_that_moved_the_most(old_centroids, CC)
    
    # if the atom hasn't moved, try again (i.e move to the top of the while loop)
    if central_atom_loc is False:
        central_atom_loc, original_circle = central_atom_loc_old, original_circle_old
        continue 
    
    # take spectrum on atom and save
    CC.ramp_bias_mV(spec_bias_mv)
    CC.stm.btn_vertspec(*central_atom_loc)
    CC.stm.vertsave()
 
    # set current atom locations as "old" atom locations for next loop
    Cim = np.array(CC.stm.scandata(1,4))
    C = CircCorral(Cim, zconst, nmx, nmy)
    C.subtract_plane()
    C.get_region_centroids(percentile=98, edge_cutoff=0.1, show=False)
    old_centroids = C.centroids

NameError: name 'Createc_Controller' is not defined